In [1]:
import os
os.environ['KERAS_BACKEND'] = 'theano'
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb
from keras.layers import Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D,BatchNormalization,Flatten
from keras.models import load_model
from keras.preprocessing.text import text_to_word_sequence

Using Theano backend.


In [2]:
# Loading IMDB data

def load_data(max_features = 100000, maxlen = 100):
    (x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
    x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
    x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
    print('x_train shape:', x_train.shape)
    print('x_test shape:', x_test.shape)
    return x_train, y_train, x_test, y_test

In [23]:
class model:
    def __init__(self, max_features = 100000,
                max_len = 100):
        self.max_features = max_features
        self.max_len = max_len
        self.words = imdb.get_word_index()
        
        pass
    def build(self):
        '''
        Build a CNN model
        '''
        embedding_dims = 50
        filters = 64
        kernel_size = 3
        hidden_dims = 250 

        epochs = 2
        number_conv_layer = 1

        self.model = Sequential()
        self.model.add(Embedding(self.max_features,
                        embedding_dims,
                        input_length=100))

        self.model.add(Conv1D(filters,
                     kernel_size,
                     padding='valid',
                     activation='relu',
                     strides=1))
        self.model.add(Activation('relu'))
        self.model.add(MaxPooling1D(pool_size=2))
        self.model.add(Dropout(0.2))
        
        self.model.add(Conv1D(filters,
                     kernel_size,
                     padding='valid',
                     activation='relu',
                     strides=1))
        self.model.add(Activation('relu'))
        self.model.add(MaxPooling1D(pool_size=2))
        self.model.add(Dropout(0.2))
        # Flatten
        self.model.add(Flatten())
        
        # We add a vanilla hidden layers
        self.model.add(Dense(16))
        self.model.add(Activation('relu'))
        self.model.add(Dropout(0.2))
        self.model.add(Dense(1))
        self.model.add(Activation('sigmoid'))
        self.model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['acc'])
        
        return model
    
    def train(self, x_train, y_train,
              x_test, y_test,
              batch_size = 32,
              epochs = 10,
              verbose = 2):
        '''
        Train model and return history
        '''
        history = self.model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test), verbose = 2)
        return history
    
    def save_model(self, file_name):
        
        self.model.save(file_name)
    
    def load_model(self, file_name):
        '''
        Load a previously trained model
        '''
        try:
            del self.model
        except: pass
        
        self.model = load_model(file_name)
        
    
    def summary(self):
        '''
        Return model summary
        '''
        self.model.summary()
    
    def _scores(self, results):

        final = []
        results = [item for sublist in results for item in sublist]
        for result in results:
            p = result * 2
            final.append(p-1)
        return final
    
    def predict_array(self, array):
        self.prediction = self._scores(self.model.predict(array))
    
    def predict_phrase(self, phrase):
        # define the document
        # tokenize the document
        
        word_vecs = []
        
        if isinstance(phrase, str):
            texts = [phrase[0:100]]
        
        if isinstance(phrase, list):
            texts = [x[0:100] for x in phrase]
            
        self.texts = texts
        for text in self.texts:
            result = text_to_word_sequence(text)
            word_vec = []
            for word in result:
                word_vec.append(self.words[word])
                
            word_vecs.append(word_vec)
        x = sequence.pad_sequences(word_vecs, maxlen= self.max_len)
        self.prediction = self._scores(self.model.predict(x))


In [24]:
x_train, y_train, x_test, y_test = load_data()

x_train shape: (25000, 100)
x_test shape: (25000, 100)


In [25]:
m = model()
m.build()
m.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 100, 50)           5000000   
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 98, 64)            9664      
_________________________________________________________________
activation_18 (Activation)   (None, 98, 64)            0         
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 49, 64)            0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 49, 64)            0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 47, 64)            12352     
_________________________________________________________________
activation_19 (Activation)   (None, 47, 64)            0         
__________

In [26]:
m.train(x_train, y_train, x_test, y_test, epochs=3)

Train on 25000 samples, validate on 25000 samples
Epoch 1/3
 - 55s - loss: 0.4711 - acc: 0.7523 - val_loss: 0.4408 - val_acc: 0.8019
Epoch 2/3
 - 91s - loss: 0.2227 - acc: 0.9159 - val_loss: 0.3902 - val_acc: 0.8364
Epoch 3/3
 - 94s - loss: 0.0947 - acc: 0.9670 - val_loss: 0.5636 - val_acc: 0.8130


In [27]:
m.save_model('sentiment_analysis.h5')

In [53]:
m.predict_phrase(['I hated this movie do not want to watch it again.', 'I loved this movie so much'])

In [54]:
m.prediction

[-0.3750063180923462, 0.2114342451095581]